In [ ]:
import pandas as pd
from difflib import SequenceMatcher

In [68]:
#Importation des datasets
df  = pd.read_csv(r"data\dataset_final_v1.csv").drop(columns="Unnamed: 0") 
actor = pd.read_csv("data//actor_scrap.csv").drop(columns="Unnamed: 0")
director = pd.read_csv(r"data\director_scrap.csv").drop(columns="Unnamed: 0")
compagnies = pd.read_csv(r'data\compagny_scrap.csv').drop(columns="Unnamed: 0")

df.head()

,date,titre,budget,recette,duree,realisateur,casting,compagnies_production,suite,Comedy,...,Crime,Romance,Animation,Mystery,War,Music,Western,History,Documentary,TV Movie
0,2014-11-12,Dumb and Dumber To,40000000,169837010,110,"['Bobby Farrelly', 'Peter Farrelly']","['Jim Carrey', 'Jeff Daniels', 'Laurie Holden'...","['New Line Cinema', 'Universal Pictures', 'Con...",3,1,...,0,0,0,0,0,0,0,0,0,0
1,2004-11-11,Марс,2000000,240000,100,['Anna Melikyan'],"['Nana Kiknadze', 'Artur Smolyaninov', 'Yevgen...","['Central Partnership', 'Studio SLON']",1,1,...,0,0,0,0,0,0,0,0,0,0
2,2014-03-20,Captain America: The Winter Soldier,170000000,714766572,136,"['Anthony Russo', 'Joe Russo']","['Chris Evans', 'Scarlett Johansson', 'Sebasti...",['Marvel Studios'],2,0,...,0,0,0,0,0,0,0,0,0,0
3,2012-03-15,Тот ещё Карлосон!,4000000,9938268,80,['Sarik Andreasyan'],"['Mikhail Galustyan', 'Igor Vernik', 'Mariya S...",['Enjoy Movies'],1,1,...,0,0,0,0,0,0,0,0,0,0
4,1997-04-03,The Saint,68000000,118063304,116,['Phillip Noyce'],"['Val Kilmer', 'Elisabeth Shue', 'Rade Šerbedž...","['Paramount', 'Mace Neufeld Productions', 'Rys...",1,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
list_keys = ['realisateur', 'casting', 'compagnies_production']

def str_to_list(df, key):
    df_new = df

    for k in key: #on parcourt les clés à modifier spécifiées en argument de la fonction

        for ind in range(df_new.shape[0]): #on parcourt toutes les lignes du Dataframe

            string = df_new[k].iloc[ind] #on récupère la valeur à modifier pour chaque ligne

            if type(string) == str : # on vérifie si c'est bien une string
                
                #création de la nouvelle variable de type liste qui remplacera l'ancienne

                liste_finale = []

                #remplissage de la variable de type liste en faisant des opérations sur la string

                list_string = string[1:-1].split(',')
                for i,j in enumerate(list_string):
                    if i == 0:
                        mot = j[1:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                    else:
                        mot = j[2:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                
                #affectation de la nouvelle valeur de type liste à l'élément modifié du Dataframe

                df_new.at[ind,k] = liste_finale

    return df_new

df = str_to_list(df, list_keys)

In [ ]:
compagnies.head()

In [ ]:
#Contruction des notes des acteurs/réalisateur
#Acteur
actor["rating"] = 100*actor["Worldwide_box_office"]/actor['Worldwide_box_office'].max()
actor["rating"] = actor["rating"].astype(int)

#Réalisateur
director["rating"] = 100*director["Worldwide_box_office"]/director['Worldwide_box_office'].max()
director["rating"] = director["rating"].astype(int)

#Compagnies
compagnies["rating"] = 100*compagnies["Worldwide_box_office"]/compagnies['Worldwide_box_office'].max()
compagnies["rating"] = compagnies["rating"].astype(int)


In [ ]:
actor = actor[actor["rating"] > 0]
director = director[director["rating"] > 0]
compagnies = compagnies[compagnies["rating"] > 0]
compagnies.head()

In [ ]:
"""actor.to_csv("data/actor_ratings.csv")
director.to_csv("data/director_ratings.csv")
compagnies.to_csv("data/compagnies_ratings.csv")"""

In [ ]:
#Création des dictionnaires
actor_dict = {}
for i, row in actor.iterrows():
    actor_dict[row.Actor_Name] = row.rating

director_dict = {}
for i, row in director.iterrows():
   director_dict[row.Director_Name] = row.rating

compagnies_dict = {}
for i, row in compagnies.iterrows():
   compagnies_dict[row.Compagny_Name] = row.rating



In [ ]:
def comparaison_string(str1, str2): #Renvoie la ressemblance en deux strings
    S = SequenceMatcher(None,  str1, str2)
    if S.ratio() > 0.8 :
        return True
    else :
        return False

def comparaison_dict(string, dict): #Renvoie le rating associé à un acteur
    for name, rating in dict.items():
        if comparaison_string(string, name):
            return rating
    return 0

In [ ]:
#On remplace maintenant les données par leur score
for i, row in df.iterrows():
    sum_actor = 0
    sum_director = 0
    sum_compagnies = 0
    for actor in row.casting:
        sum_actor += comparaison_dict(actor, actor_dict)
    for director in row.realisateur:
        sum_director += comparaison_dict(director, director_dict)
    for compagny in row.compagnies_production:
        sum_compagnies += comparaison_dict(compagny, compagnies_dict)
    
    df.loc[i, 'casting_score'] = int(sum_actor)
    df.loc[i, 'realisateur_score'] = int(sum_director)
    df.loc[i, 'compagny_score'] = int(sum_compagnies)
df.to_csv("arrange.csv")

In [ ]:
df2  = pd.read_csv(r"data\arrange.csv").drop(columns="Unnamed: 0") 
df2.head()

In [ ]:
df2[df2["casting_score"] == df2.casting_score.min()]

In [ ]:
df2[df2["realisateur_score"] == df2.realisateur_score.max()]

In [ ]:
df2[df2["compagny_score"] == df2.compagny_score.max()]

In [65]:
from dateutil import parser
import datetime
from dateutil.relativedelta import relativedelta
import math


In [70]:
df ["date"] = pd.to_datetime(df["date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4695 entries, 0 to 4694
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   4695 non-null   datetime64[ns]
 1   titre                  4695 non-null   object        
 2   budget                 4695 non-null   int64         
 3   recette                4695 non-null   int64         
 4   duree                  4695 non-null   int64         
 5   realisateur            4695 non-null   object        
 6   casting                4695 non-null   object        
 7   compagnies_production  4695 non-null   object        
 8   suite                  4695 non-null   int64         
 9   Comedy                 4695 non-null   int64         
 10  Drama                  4695 non-null   int64         
 11  Horror                 4695 non-null   int64         
 12  Thriller               4695 non-null   int64         
 13  Act

In [75]:
for i, row in df.iterrows():
    df2 = df[df["date"] > df.loc[i, "date"] - relativedelta(months=1)]
    df2 = df2[df2["date"] < df.loc[i, "date"] + relativedelta(months=1)]
    nb = -1
    nb_genres = 0
    for index in range(9,28):
        if row[index] == 1:
            nb_genres += 1

    for k, row2 in df2.iterrows() :
        nb_commun = 0
        for index in range(9,28):
            if row[index] == 1 and row[index] == row2[index]:
                nb_commun += 1
        
        if nb_commun >= math.ceil(nb_genres*0.5):
            nb  += 1


    df.loc[i, "nb_films_concu"] = nb

df.describe()

,budget,recette,duree,suite,Comedy,Drama,Horror,Thriller,Action,Adventure,...,Romance,Animation,Mystery,War,Music,Western,History,Documentary,TV Movie,nb_films_concu
count,4.695000e+03,4.695000e+03,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,...,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000,4695.000000
mean,3.319896e+07,9.371616e+07,111.790841,1.277742,0.347817,0.476251,0.105644,0.264111,0.278807,0.175719,...,0.176358,0.060490,0.089670,0.035570,0.032588,0.010863,0.056443,0.010650,0.000852,8.108626
std,4.239805e+07,1.861755e+08,21.678317,1.225860,0.476328,0.499489,0.307415,0.440906,0.448460,0.380621,...,0.381165,0.238418,0.285739,0.185235,0.177574,0.103667,0.230800,0.102657,0.029179,6.973298
min,3.000000e+05,2.000000e+05,63.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000e+06,5.993693e+06,97.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,1.800000e+07,2.666720e+07,107.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,4.000000e+07,9.702621e+07,123.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000
max,3.800000e+08,2.920357e+09,339.000000,34.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,43.000000
